In [125]:
%pylab inline
import os
import pandas as pd
from shutil import copyfile, rmtree
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.layers import Flatten, Dense, Dropout, BatchNormalization
from keras.models import Model
from keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from keras import optimizers
import scipy.io
from PIL import Image
from sklearn import metrics

Populating the interactive namespace from numpy and matplotlib


In [2]:
def dynamic_std_print(string_to_print):
    """
    Printing string on standard output and refreshing output line
    """
    sys.stdout.write('\r')
    sys.stdout.write(string_to_print)
    sys.stdout.flush()

# 1) Get data

Gather our files paths

In [3]:
# Get local path
local_path = os.getcwd()
pictures_path = local_path + '/data/images/'

In [4]:
# List images folders
breeds_folders = [local_path + "/data/images/" + file_name for file_name in os.listdir(local_path + "/data/images") if os.path.isdir(local_path + "/data/images/" + file_name)]

In [105]:
# List breeds id and name
breeds_id = [breed_path.split("/")[-1].split("-")[0] for breed_path in breeds_folders]
breeds_name = ["-".join(breed_path.split("/")[-1].split("-")[1:]) for breed_path in breeds_folders]
breed_dict = dict(zip(breeds_id, breeds_name))
n_breeds = len(breeds_name)

Use train and test sets provided with data

In [6]:
train_mat = scipy.io.loadmat(local_path + '/data/lists/train_list.mat')
test_mat = scipy.io.loadmat(local_path + '/data/lists/test_list.mat')

Convert matlab matrix to pandas dataframe

In [119]:
# For train set
train_df = pd.DataFrame(train_mat['labels'], columns=['label'])
train_df['file'] = [str(file_name[0].tolist()[0]) for file_name in train_mat['file_list']]
train_df['breed_id'] = train_df.file.apply(lambda x: x.split("/")[0].split("-")[0])
train_df['breed_name'] = train_df.breed_id.apply(lambda x: breed_dict[x])

# For test set
test_df = pd.DataFrame(test_mat['labels'], columns=['label'])
test_df['file'] = [str(file_name[0].tolist()[0]) for file_name in test_mat['file_list']]
test_df['breed_id'] = test_df.file.apply(lambda x: x.split("/")[0].split("-")[0])
test_df['breed_name'] = test_df.breed_id.apply(lambda x: breed_dict[x])

In [121]:
# Get label-names dict
breed_labels_df = train_df.loc[:, ['label', 'breed_name']].drop_duplicates()
breed_label_dict = dict(zip(breed_labels_df.label.values, breed_labels_df.breed_name.values))
label_breed_dict = dict([[v,k] for k,v in breed_label_dict.items()])

In [122]:
train_df.sample(5)

,label,file,breed_id,breed_name
8446,85,n02106662-German_shepherd/n02106662_1637.jpg,n02106662,German_shepherd
2203,23,n02091244-Ibizan_hound/n02091244_1917.jpg,n02091244,Ibizan_hound
8324,84,n02106550-Rottweiler/n02106550_12760.jpg,n02106550,Rottweiler
5014,51,n02097658-silky_terrier/n02097658_4363.jpg,n02097658,silky_terrier
3153,32,n02093754-Border_terrier/n02093754_175.jpg,n02093754,Border_terrier


Copy files to train and test folders for CNN training

In [9]:
# # Copy train files to train folder
# for file in train_df.file.values:
#     folder = file.split("/")[0]
#     file_name = file.split("/")[-1]
#     if not os.path.exists(local_path + '\\data\\train\\' + folder):
#         os.mkdir(local_path + '\\data\\train\\' + folder)
#         dynamic_std_print("Creating folder {}                  ".format(folder))
#     copyfile(pictures_path + file, local_path + '\\data\\train\\' + folder + '\\' + file_name)
    
# # Copy test files to test folder
# for file in test_df.file.values:
#     folder = file.split("/")[0]
#     file_name = file.split("/")[-1]
#     if not os.path.exists(local_path + '\\data\\test\\' + folder):
#         os.mkdir(local_path + '\\data\\test\\' + folder)
#         dynamic_std_print("Creating folder {}                  ".format(folder))
#     copyfile(pictures_path + file, local_path + '\\data\\test\\' + folder + '\\' + file_name)

Copy files to small train and test folders for CNN training (few files for debuging purposes)

In [90]:
# Remove folders content
folders = [local_path + '\\data\\debuging\\train\\',
           local_path + '\\data\\debuging\\test\\test_folder',
           local_path + '\\data\\debuging\\valid\\']
for folder in folders:
    for the_file in os.listdir(folder):
        file_path = os.path.join(folder, the_file)
        try:
            if os.path.isfile(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                rmtree(file_path)
        except Exception as e:
            print(e)

[WinError 32] Le processus ne peut pas accéder au fichier car ce fichier est utilisé par un autre processus: 'D:\\FORMATIONS\\OCLR\\0-PARCOURS-DATA_SCIENTIST\\6-Realiser_Des_Indexations_Automatiques_d_Images\\data\\debuging\\test\\test_folder\\n02091467_3942.jpg'


In [91]:
# Copy train and valid  files to train and valid folders
# Get n pic for each label
n_train = 10
n_valid = 2
n_test = 2

for label in train_df.label.unique():
    label_df = train_df[train_df.label == label].sample(n_train + n_valid, random_state=0)
    
    # Copy training files
    for file in label_df.file.values[:n_train]:
        folder = file.split("/")[0]
        file_name = file.split("/")[-1]
        if not os.path.exists(local_path + '\\data\\debuging\\train\\' + folder):
            os.mkdir(local_path + '\\data\\debuging\\train\\' + folder)
            dynamic_std_print("Creating folder {}                             ".format(folder))
        copyfile(pictures_path + file, local_path + '\\data\\debuging\\train\\' + folder + '\\' + file_name)
        
    # Copy validation files        
    for file in label_df.file.values[n_train:]:
        folder = file.split("/")[0]
        file_name = file.split("/")[-1]
        if not os.path.exists(local_path + '\\data\\debuging\\valid\\' + folder):
            os.mkdir(local_path + '\\data\\debuging\\valid\\' + folder)
            dynamic_std_print("Creating folder {}                             ".format(folder))
        copyfile(pictures_path + file, local_path + '\\data\\debuging\\valid\\' + folder + '\\' + file_name)
    
# Copy test files to test folder
# Get m pic for each label
target_names = []
for label in train_df.label.unique():
    label_df = test_df[test_df.label == label].sample(n_test, random_state=0)
    for file in label_df.file.values:
        folder = file.split("/")[0]
        file_name = file.split("/")[-1]
        file_breed = file_name.split("_")[0]
        target_names.append(file_breed)
        copyfile(pictures_path + file, local_path + '\\data\\debuging\\test\\test_folder\\' + file_name)

Creating folder n02116738-African_hunting_dog                                        

# 2) Modify Conventional Neural Network for Transfer Learning

We will use Keras VGG-16 CNN already trained on ImageNet database in order to predict our dogs breeds.

Steps are as follow :
- 1) Call the pre-trained VGG-16 CNN
- 2) Apply partial fine-tuning because we don't have much data (20e3) and they are dogs only, unlike ImageNet database
- 3) Run modified model for dogs classification

#### Prepare pictures batches

Training batch

In [12]:
train_batch_size = 20
# Initialise data generator
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
# Fit generator from train folder. Folder must contains one sub-directory by class
train_generator = train_datagen.flow_from_directory(local_path + '/data/debuging/train',
                                                    target_size=(224,224),
                                                    color_mode='rgb',
                                                    batch_size=train_batch_size,
                                                    class_mode='categorical',
                                                    shuffle=True)

Found 1200 images belonging to 120 classes.


Validation batch

In [13]:
valid_batch_size = 20
# Initialise data generator
valid_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
# Fit generator from train folder. Folder must contains one sub-directory by class
valid_generator = valid_datagen.flow_from_directory(local_path + '/data/debuging/valid',
                                                    target_size=(224,224),
                                                    color_mode='rgb',
                                                    batch_size=valid_batch_size,
                                                    class_mode='categorical',
                                                    shuffle=True)

Found 240 images belonging to 120 classes.


Test batch

In [25]:
test_batch_size = 20
# Initialise data generator
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
# Fit generator from train folder. Folder must contains one sub-directory by class
test_generator = test_datagen.flow_from_directory(local_path + '/data/debuging/test',
                                                  target_size=(224,224),
                                                  color_mode='rgb',
                                                  batch_size=test_batch_size,
                                                  class_mode='categorical',
                                                  shuffle=True)

Found 240 images belonging to 1 classes.


In [14]:
# count=1
# for item in test_generator:
#     print(count)
# #     save_test_item = item
# #     break
#     count+=1

In [15]:
# save_test_item[1]

#### Prep model

In [16]:
# Charger VGG-16 pré-entraîné sur ImageNet et sans les couches fully-connected
model = VGG16(weights="imagenet", include_top=False, input_shape=(224, 224, 3))

# Récupérer la sortie de ce réseau
x = model.output

# ???
x = model.output
x = Flatten()(x)
x = Dropout(0.4)(x)
# let's add two fully-connected layer
x = Dense(2048, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.4)(x)
x = Dense(2048, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.2)(x)

# Ajouter la nouvelle couche fully-connected pour la classification à 120 classes
predictions = Dense(120, activation='softmax')(x)

# Définir le nouveau modèle
new_model = Model(inputs=model.input, outputs=predictions)

In [17]:
# Ne pas entraîner les 5 premières couches (les plus basses) 
for layer in new_model.layers[:5]:
   layer.trainable = False

In [18]:
# Compiler le modèle 
new_model.compile(loss="categorical_crossentropy", optimizer=optimizers.SGD(lr=0.0001, momentum=0.9), metrics=["accuracy"])

What's in our model so far ?

In [19]:
new_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [20]:
#Entraîner sur les données d'entraînement (X_train, y_train)
model_complete = new_model.fit_generator(train_generator,
#                                          steps_per_epoch=2000,
                                         epochs=3,
                                         validation_data=valid_generator,
#                                          validation_steps=800,
                                         verbose=1)

Epoch 1/3
60/60 [==============================] - 737s 12s/step - loss: 5.7893 - acc: 0.0092 - val_loss: 5.1261 - val_acc: 0.0083
Epoch 2/3
60/60 [==============================] - 737s 12s/step - loss: 5.6708 - acc: 0.0125 - val_loss: 9.1359 - val_acc: 0.0167
Epoch 3/3
60/60 [==============================] - 745s 12s/step - loss: 5.8841 - acc: 0.0125 - val_loss: 5.8571 - val_acc: 0.0125


Predict on test pictures

In [35]:
pred = new_model.predict_generator(test_generator, verbose=1)

12/12 [==============================] - 54s 5s/step


Get max proba for each picture (most probable class i.e. dog's breed)

In [123]:
# get max proba
max_preds = pred.argmax(axis=1)
# convert to dataframe
pred_df = pd.DataFrame(max_preds, columns=['pred_label'])
pred_df['pred_name'] = pred_df.pred_label.apply(lambda x: breed_label_dict[x])
pred_df['true_id'] = target_names
pred_df['true_name'] = pred_df.true_id.apply(lambda x: breed_dict[x])
pred_df['true_label'] = pred_df.true_name.apply(lambda x: label_breed_dict[x])

In [127]:
pred_df.sample(5)

,pred_label,pred_name,true_id,true_name,true_label
46,1,Chihuahua,n02091467,Norwegian_elkhound,24
104,29,Staffordshire_bullterrier,n02098286,West_Highland_white_terrier,53
214,103,pug,n02112018,Pomeranian,108
76,98,Eskimo_dog,n02095570,Lakeland_terrier,39
183,103,pug,n02108089,boxer,92


Get accuracy

In [126]:
metrics.accuracy_score(pred_df.true_label, pred_df.pred_label)

0.0125

In [23]:
# test_pic = test_df.sample(1)
# pic_path = pictures_path + test_pic.file.values

# img = load_img(pic_path.tolist()[0], target_size=(224, 224))  # Charger l'image
# img = img_to_array(img)  # Convertir en tableau numpy
# img = img.reshape((1, img.shape[0], img.shape[1], img.shape[2]))  # Créer la collection d'images (un seul échantillon)
# img = preprocess_input(img)  # Prétraiter l'image comme le veut VGG-16

In [28]:
# y = new_model.predict(img)  # Prédir la classe de l'image (parmi les 1000 classes d'ImageNet)

In [22]:
# model_complete = new_model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=2)